# Présentation des données

In [78]:
# Exo 1 - Préparation des données 

# importation des données
import pandas as pd 
import numpy as np 
# import du fichier
data = pd.read_csv("synthetic.csv")

# Visualisation des données
print(data.head())


      Attr_A     Attr_B       Attr_C      Attr_D      Attr_E       Attr_F  \
0  12.478064  14.784992  1247.541877  100.962061   52.462177  1089.398211   
1   8.264345   8.854181  1389.686814   99.526529   65.106526  1186.523399   
2   9.875571  13.292442   779.077401  123.993772  104.699796  1201.722480   
3   9.207661   9.346913   965.468523   89.176009  102.628284   743.913507   
4   8.863842  12.542969  1096.386230  106.595385  131.813380   883.059615   

      Attr_G     Attr_H     Attr_I      Attr_J       Attr_K     Attr_L  \
0  10.575834   8.375407  10.288159  110.746551   994.367610   9.069350   
1   9.500485  10.088058   9.371983   78.210274   943.089589   9.988919   
2   9.545266  14.266238   9.703551   86.252483  1082.989190  10.084217   
3   9.777953  11.613946   8.912059   96.727873   812.800511   8.621781   
4  10.092974  13.556029  11.649982   21.566576   971.083175  10.072271   

        Attr_M      Attr_N  Class  
0  1027.953917  109.672758      1  
1  1120.317724   83.

In [ ]:
# 1 - Nombre de colonnes (attributs) dans le DataFrame
num_attributes = data.shape[1]

# Afficher le nombre d'attributs
print(f"Le nombre d'attributs dans le fichier est : {num_attributes}")


In [ ]:
# Type de données et valeurs manquantes
print(data.info())

In [ ]:
# Avoir le nombre d'attributs dans le modèle
print(data.columns)
# 14 attributs dans le modèle

In [ ]:
# Obtenir les classes uniques dans la colonne 'Class'
classes_uniques = data['Class'].unique()

# Nombre de classes différentes
num_classes = len(classes_uniques)

# Afficher le nombre de classes différentes
print(f"Le nombre de classes différentes dans les données est : {num_classes}")

In [ ]:
# combien d'instances compte chaque classe?
nbr_instances = data['Class'].value_counts()
print(nbr_instances)

# Sortie 
# Class
# 1    908
# 0    674
# 2    472
# 3    244
# Name: count, dtype: int64

# Les données sont-elles linéairement séparables ?
Non, si on observe le schéma 1 on voit que les données ne le sont pas.
De plus si l'on choisit de les ranger par classe , on peut s'apercevoir que 


In [ ]:
import matplotlib.pyplot as plt # import biblio matplot
plt.figure(figsize=(10, 6))
plt.scatter(data['Attr_A'], data['Attr_B'], c=data['Class'], alpha=0.5, cmap='viridis')
plt.xlabel('Attribut 1')
plt.ylabel('Attribut 2')
plt.title('Scatter Plot des attributs par classe')
plt.colorbar(label='Classe')
plt.show()

# On peut voir clairement que ce n'est pas divisible linéairement à l'état brut
# je pense que use image est vraiment mieux


## 5 et 6 (voir compte-rendu.md) 

# 2 Mise en oeuvre des modèles

In [ ]:
# Choisir un attribut à analyser, par exemple 'Attr_A'
attribute = 'Attr_A'


# Calculer les quartiles pour l'attribut choisi
quartiles = data[attribute].quantile([0.25, 0.5, 0.75])

# Sort the attribute values and print them
sorted_attribute = data[attribute].sort_values()
print(sorted_attribute)
print(quartiles)
# Afficher les quartiles
print(f"Quartile 1 (Q1) de l'attribut '{attribute}': {quartiles[0.25]}")
print(f"Médiane (Q2) de l'attribut '{attribute}': {quartiles[0.5]}")
print(f"Quartile 3 (Q3) de l'attribut '{attribute}': {quartiles[0.75]}")


# 6 : 

# Partie 2 : Mise en oeuvre des modèles.

# Arbre de décision 


In [93]:
# Arbre de décision

# Calcul de l'entropie

"""
L'entropie est une mesure de l'incertitude associée à une variable aléatoire.
"""

def entropie(dataframe , attribut_cible):  
    # Calcul de la probabilité de chaque classe
    compte_classe = dataframe[attribut_cible].value_counts()
    #print(compte_classe)
    proba = compte_classe / compte_classe.sum()
    #print(proba) 
    # Calcul de l'entropie
    entropie = - (proba * np.log2(proba+ np.finfo(float).eps)).sum() # éviter log2(0)
    return entropie

# Test de la fonction
print(entropie(data, 'Attr_A'))


11.166163082645376


11.166163082646115
11.166163082645376

11.166163082646115
11.166163082645376

11.166163082646115
11.166163082645376

1.8608867211835993
1.860886721183598

In [ ]:
import pandas as pd
import numpy as np

# Fonction pour calculer tous les quartiles d'un attribut donné
def calculate_quartiles(data, attribute):
    quartiles =data[attribute].quantile([0.25, 0.5, 0.75])
    return quartiles

# Test de la fonction sur le DataFrame chargé

print(calculate_quartiles(data, 'Attr_A'))


In [ ]:
data

In [ ]:
data.sort_values(by="Attr_C")

In [ ]:
# data.head()
sorted = data.sort_values(by="Attr_A")
print(len(sorted))

In [ ]:
def gain_information(dataframe, attribut_cible, attribut_test):
    """
    Calculate the information gain from splitting the data based on a test attribute.

    Parameters:
    dataframe (pd.DataFrame): The DataFrame containing the data to partition.
    attribut_cible (str): The target attribute we want to predict.
    attribut_test (str): The attribute whose gain we want to calculate.

    Returns:
    tuple: A tuple containing:
        - attribut_test (str): The test attribute.
        - max_gain (float): The maximum information gain obtained.
        - best_split_value (float): The split value that provides the best gain.
        - best_partitions (tuple): A tuple containing two DataFrames representing the lower and upper partitions
          resulting from the best split.
    """
    
    # Initial entropy of the target attribute
    entropie_initiale = entropie(dataframe, attribut_cible)

    # The gain, split_value and partitions initialized
    max_gain = 0
    best_split_value = None
    best_partitions = None

    # Check for no unique values in the attribute being tested
    if len(dataframe[attribut_test].unique()) <= 1:
        return None

    # Sorting data by the attribute to test
    sorted_data = dataframe.sort_values(by=attribut_test)

    # Unique values of the attribute to test, considering quartiles to reduce complexity
    quartiles = calculate_quartiles(sorted_data, attribut_test).to_list()

    # Adding the min and max values to cover the entire range of the attribute
    # quartiles = [sorted_data[attribut_test].min()] + quartiles + \
    #     [sorted_data[attribut_test].max()]
    # Voir si je n'enlève pas min et max valeur

    # Iterating through the sorted unique values to find the best split
    for split_value in quartiles:
        # Partitioning the data based on the split value
        lower_partition = sorted_data[sorted_data[attribut_test] < split_value]
        upper_partition = sorted_data[sorted_data[attribut_test]
                                      >= split_value]

        # Calculating the weighted entropy for the partitions
        # Row counts.
        total_instances = len(sorted_data)
        lower_weight = len(lower_partition) / total_instances
        upper_weight = len(upper_partition) / total_instances

        # Computing the weighted_entropy
        weighted_entropy = (lower_weight * entropie(lower_partition, attribut_cible)) + \
                           (upper_weight * entropie(upper_partition, attribut_cible))

        # Information gain for the current split
        current_gain = entropie_initiale - weighted_entropy

        # If the current gain is greater than the max_gain, update max_gain and best_split_value
        if current_gain > max_gain:
            max_gain = current_gain
            best_split_value = split_value
            best_partitions = (lower_partition, upper_partition)

    # Returning the attribute, gain, split_value, and partitions as a tuple
    return attribut_test, max_gain, best_split_value, best_partitions


# Testing the function with an example attribute
# Let's use 'Attr_A' as the attribute to test and 'Class' as the target
test_gain_info = gain_information(data, 'Class', 'Attr_H')
test_gain_info

In [ ]:
def calculate_quartiles_opti(data, attribute):
    quartiles =data[attribute].quantile([0.25, 0.5, 0.75])
    return quartiles.tolist()

def check_variance(dataframe, attribut_test, threshold=0.01):
    return dataframe[attribut_test].var() >= threshold


def gain_information_optimized(dataframe, attribut_cible, attribut_test):
    # Si l'attribut a très peu de valeurs uniques, éviter de l'utiliser pour le split
    if dataframe[attribut_test].nunique() <= 1:
        return attribut_test, 0, None, None

    # vérifier variance     
    if not check_variance(dataframe, attribut_test):
        return attribut_test, 0, None, None


    # Tri des données par l'attribut à tester 
    sorted_data = dataframe.sort_values(by=attribut_test)
    total_count = len(sorted_data)
    entropie_initiale = entropie(dataframe, attribut_cible)

    max_gain = 0
    best_split_value = None
    best_partitions = None

    # Calcul des quartiles
    quartiles = calculate_quartiles_opti(sorted_data, attribut_test)
    thresholds = quartiles 

    # Initialisation des partitions
    lower_partition = pd.DataFrame(columns=sorted_data.columns)
    upper_partition = sorted_data.copy()

    previous_threshold = thresholds[0]

    for threshold in thresholds[1:]:
        # Déplacer les données de upper à lower basées sur le seuil actuel
        mask = (upper_partition[attribut_test] < threshold)
        moving_data = upper_partition[mask]
        lower_partition = pd.concat([lower_partition, moving_data])
        upper_partition = upper_partition[~mask]

        # Vérifier si les partitions sont vides
        if lower_partition.empty or upper_partition.empty:
            continue

        # Calcul de l'entropie pondérée pour les partitions actuelles
        lower_weight = len(lower_partition) / total_count
        upper_weight = len(upper_partition) / total_count

        weighted_entropy = (lower_weight * entropie(lower_partition, attribut_cible) +
                            upper_weight * entropie(upper_partition, attribut_cible))
        current_gain = entropie_initiale - weighted_entropy

        # Mise à jour du meilleur gain si nécessaire
        if current_gain > max_gain:
            max_gain = current_gain
            best_split_value = threshold
            best_partitions = (lower_partition.copy(), upper_partition.copy())

    return attribut_test, max_gain, best_split_value, best_partitions



# Testing the function with an example attribute
# Let's use 'Attr_A' as the attribute to test and 'Class' as the target
test_gain_info = gain_information_optimized(data, 'Class', 'Attr_H')
test_gain_info

In [ ]:
def find_best_gain(dataframe, attribut_cible):
    """
    Calculate the best information gain and corresponding split in a DataFrame.
    
    Parameters:
    dataframe (pd.DataFrame): The input data as a pandas DataFrame.
    attribut_cible (str): The target attribute that we want to predict (e.g. 'Class').
    
    Returns:
    tuple: A tuple containing the following elements:
        - best_attribute (str): The attribute that yields the best information gain.
        - best_gain (float): The highest information gain observed.
        - best_split_value (float): The split value that produces the best gain.
        - best_partitions (tuple): A tuple containing the two partitions resulting from the best split.
    """
    # Initialize variables to track the best gain and the associated attribute
    best_gain = 0
    best_attribute = None
    best_split_value = None
    best_partitions = None

    # Iterate over all the attributes in the DataFrame, except the target attribute
    for test_attribute in dataframe.columns:
        if test_attribute == attribut_cible:
            continue  # Skip the target attribute

        # Calculate the information gain for the current attribute
        # result = gain_information(dataframe, attribut_cible, test_attribute)
        
        # optimized version
        result = gain_information(dataframe, attribut_cible, test_attribute)
        
        # If the result is None, skip to the next attribute
        if result is None:
            continue

        # Unpack the result from gain_information
        _, current_gain, split_value, partitions = result

        # Update the variables if the current gain is higher than the best gain
        if current_gain > best_gain:
            best_gain = current_gain
            best_attribute = test_attribute
            best_split_value = split_value
            best_partitions = partitions

    # Return the best attribute, gain, split value, and partitions
    return best_attribute, best_gain, best_split_value, best_partitions


find_best_gain(data, 'Class')

In [ ]:
from sklearn.model_selection import train_test_split

def split_data(dataframe, attribut_cible, test_size=0.2, random_state=42):
    """
    Splits the dataframe into training and testing sets.

    Parameters:
    dataframe (pd.DataFrame): The DataFrame containing the data to split.
    attribut_cible (str): The target attribute we want to predict. e.g ("Class")
    test_size (float): The proportion of the data to include in the test split. Default is 0.2.
    random_state (int): Controls the shuffling applied to the data before applying the split. Default is 42.

    Returns:
    tuple: A tuple containing:
        - X_train (pd.DataFrame): The training features.
        - X_test (pd.DataFrame): The testing features.
        - y_train (pd.Series): The training target attribute.
        - y_test (pd.Series): The testing target attribute.
    """
    # Separate features and target attribute
    X = dataframe.drop(columns=[attribut_cible])
    y = dataframe[attribut_cible]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test =  split_data(data, "Class")

In [ ]:
class DecisionNode:
    def __init__(
            self,
            is_leaf,
            attribute=None,
            split_value=None,
            left=None,
            right=None,
            prediction=None):
        """
        Initialize a decision tree node.

        Parameters:
        is_leaf (bool): Whether the node is a leaf node.
        attribute (str, optional): The attribute to split on if the node is not a leaf.
        split_value (float, optional): The split value for the attribute if the node is not a leaf.
        left (DecisionNode, optional): The left child node.
        right (DecisionNode, optional): The right child node.
        value (object, optional): The target value if the node is a leaf.
        """
        self.is_leaf = is_leaf
        self.attribute = attribute
        self.split_value = split_value
        self.left = left
        self.right = right
        self.prediction = prediction

    def _is_leaf(self):
        return self.is_leaf

In [ ]:
class DecisionTree:
    """
    Represents a decision tree.

    Attributes:
    max_depth (int): The maximum depth of the tree.
    tree (DecisionNode): The root node of the tree.
    """

    def __init__(self, max_depth=8):
        """
        Initialize the decision tree.

        Parameters:
        max_depth (int, optional): The maximum depth of the tree. Default is 8.
        """
        self.max_depth = max_depth
        self.tree = None

    def _build_tree(self, data, target_attribute, depth=0):
        """
        Recursively build the decision tree.

        Parameters:
        data (pd.DataFrame): The input data as a pandas DataFrame.
        target_attribute (str): The target attribute that we want to predict.
        depth (int): The current depth of the tree.

        Returns:
        DecisionNode: The root node of the decision tree.
        """
        # Check stopping conditions: maximum depth or pure leaf
        if depth >= self.max_depth:
            # Return a leaf node with the most frequent target value
            prediction = data[target_attribute].mode()[0]
            return DecisionNode(is_leaf=True, prediction=prediction)

        # Check if the data is pure (all target values are the same)
        if data[target_attribute].nunique() == 1:
            # Return a leaf node with the unique target value
            prediction = data[target_attribute].iloc[0]
            return DecisionNode(is_leaf=True, prediction=prediction)

        # Find the best attribute, gain, split value, and partitions using find_best_gain
        best_attribute, best_gain, best_split_value, best_partitions = find_best_gain(
            data, target_attribute)

        # Check if no gain is found, return the most frequent target value as a leaf node
        if best_attribute is None or best_gain <= 0:
            prediction = data[target_attribute].mode()[0]
            return DecisionNode(is_leaf=True, prediction=prediction)

        # Create the decision node with the best attribute and split value
        left_data, right_data = best_partitions
        left_child = self._build_tree(left_data, target_attribute, depth + 1)
        right_child = self._build_tree(right_data, target_attribute, depth + 1)

        # Return the decision node with the children
        return DecisionNode(
            is_leaf=False,
            attribute=best_attribute,
            split_value=best_split_value,
            left=left_child,
            right=right_child
        )

    def fit(self, data, target_attribute):
        """
        Build the decision tree based on the provided data and target attribute. e.g ("Class")
        Parameters:
        data (pd.DataFrame): The input data as a pandas DataFrame.
        target_attribute (str): The target attribute that we want to predict.
        """
        self.tree = self._build_tree(data, target_attribute)

    def train(self, dataframe, target_attribute):
        """
        Train the decision tree using the given data.

        Parameters:
        dataframe (pd.DataFrame): The DataFrame containing the training data.
        target_attribute (str): The target attribute we want to predict.
        """
        self.fit(dataframe, target_attribute)

    def predict(self, data):
        """
        Predict target attribute values for the given data using the decision tree.

        Parameters:
        data (pd.DataFrame): The data for which predictions are to be made.

        Returns:
        np.array: The predicted values of the target attribute.
        """
        predictions = data.apply(self._predict_single, axis=1)
        return predictions

    def _predict_single(self, row):
        """
        Predict the target attribute value for a single data point.

        Parameters:
        row (pd.Series): The data point as a pandas Series.

        Returns:
        object: The predicted value of the target attribute.
        """
        node = self.tree

        # Traverse the tree until a leaf node is reached
        while not node.is_leaf:
            attribute = node.attribute
            split_value = node.split_value

            if row[attribute] < split_value:
                node = node.left
            else:
                node = node.right

        # Return the value of the leaf node
        return node.prediction

    def print_tree(self, node=None, indent=""):
        """
        Print the decision tree in a human-readable way.

        Parameters:
        node (DecisionNode, optional): The current node to print. If not specified, starts with the root node.
        indent (str): Indentation for nested levels in the tree.
        """
        # If no node is specified, start with the root node
        if node is None:
            node = self.tree

        # Check if the current node is a leaf node
        if node._is_leaf():
            print(f"{indent}Leaf: Predict {node.prediction}")
        else:
            # Print the split condition and value at the current node
            print(f"{indent}Node: {node.attribute} < {node.split_value}")

            # Recursively print the left and right children
            print(f"{indent}Left:")
            self.print_tree(node.left, indent + "    ")

            print(f"{indent}Right:")
            self.print_tree(node.right, indent + "    " )

## Class qui permet d'évaluer notre model.

In [ ]:
class EvaluationMetrics:
    """
    Classe pour calculer les métriques d'évaluation.
    """
    @staticmethod
    def accuracy_score(y_true, y_pred):
        """
        Calcule l'exactitude entre les valeurs réelles et prédites.
        
        Arguments:
        y_true -- Les valeurs réelles (cibles).
        y_pred -- Les valeurs prédites par le modèle.
        
        Retourne:
        float -- L'exactitude.
        """
        correct_predictions = sum(y_true == y_pred)
        total_predictions = len(y_true)
        return correct_predictions / total_predictions

    @staticmethod
    def precision_score(y_true, y_pred, positive_label):
        """
        Calcule la précision pour une classe positive spécifiée.
        
        Arguments:
        y_true -- Les valeurs réelles (cibles).
        y_pred -- Les valeurs prédites par le modèle.
        positive_label -- La classe positive pour laquelle calculer la précision.
        
        Retourne:
        float -- La précision.
        """
        true_positives = sum((y_true == positive_label) & (y_pred == positive_label))
        predicted_positives = sum(y_pred == positive_label)
        if predicted_positives == 0:
            return 0.0
        return true_positives / predicted_positives

    @staticmethod
    def recall_score(y_true, y_pred, positive_label):
        """
        Calcule le rappel pour une classe positive spécifiée.
        
        Arguments:
        y_true -- Les valeurs réelles (cibles).
        y_pred -- Les valeurs prédites par le modèle.
        positive_label -- La classe positive pour laquelle calculer le rappel.
        
        Retourne:
        float -- Le rappel.
        """
        true_positives = sum((y_true == positive_label) & (y_pred == positive_label))
        actual_positives = sum(y_true == positive_label)
        if actual_positives == 0:
            return 0.0
        return true_positives / actual_positives

    @staticmethod
    def f1_score(y_true, y_pred, positive_label):
        """
        Calcule le score F1 pour une classe positive spécifiée.
        
        Arguments:
        y_true -- Les valeurs réelles (cibles).
        y_pred -- Les valeurs prédites par le modèle.
        positive_label -- La classe positive pour laquelle calculer le score F1.
        
        Retourne:
        float -- Le score F1.
        """
        precision = EvaluationMetrics.precision_score(y_true, y_pred, positive_label)
        recall = EvaluationMetrics.recall_score(y_true, y_pred, positive_label)
        
        if precision == 0 and recall == 0:
            return 0.0
        
        return 2 * (precision * recall) / (precision + recall)


In [ ]:
def train_and_evaluate_decision_trees(X_train, y_train, X_test, y_test, depths):
    """
    Train decision tree models with different maximum depths and evaluate their performance.

    Args:
        X_train: Training features (data).
        y_train: Training target (labels).
        X_test: Testing features (data).
        y_test: Testing target (labels).
        depths (list): List of maximum depths to test for decision trees.

    Returns:
        List of tuples containing the accuracy scores and corresponding models.
        The list is sorted in descending order based on accuracy scores.
    """
    # Combine features and labels for training and testing sets
    train_data = pd.concat([X_train, y_train], axis=1)
    test_data = pd.concat([X_test, y_test], axis=1)

    # List to store the models and their accuracy scores
    models_and_scores = []

    # Train and evaluate models with different maximum depths
    for max_depth in depths:
        # Create a decision tree classifier with the specified maximum depth
        model = DecisionTree(max_depth=max_depth)
        
        # Train the model on the combined training set
        model.train(train_data, y_train.name)  # Pass the name of the target attribute
        
        # Predict on the combined test set
        y_pred = model.predict(test_data)
        
        # Calculate the accuracy of the model using the provided y_test
        accuracy = EvaluationMetrics.accuracy_score(y_test, y_pred)
        
        # Append the accuracy score and model to the list
        models_and_scores.append((accuracy, model))
        
        # Print the results for the current model
        print(f"Max Depth: {max_depth}, Accuracy: {accuracy:.4f}")
    
    # Sort the list of models and scores in descending order based on accuracy
    models_and_scores.sort(key=lambda x: x[0], reverse=True)
    
    return models_and_scores

In [ ]:
target_attribute  = "Class"
X_train, X_test, y_train, y_test = split_data(data, target_attribute)
models_and_scores = train_and_evaluate_decision_trees(X_train, y_train, X_test, y_test, [3, 4, 5, 6, 7, 8])
print(models_and_scores)

## Two best models

In [ ]:
best_model = models_and_scores[0]
second_best_model = models_and_scores[1]

print("Best model", best_model)
print("Second Best model", second_best_model)

## Test of DecisionNode and DecisionTree class

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the target attribute
target_attribute = "Class"

# Split the data into training and testing sets using the split_data function
X_train, X_test, y_train, y_test = split_data(data, target_attribute)

# Instantiate the DecisionTree class
decision_tree = DecisionTree(max_depth=8)
evaluation_metric = EvaluationMetrics()

# Train the decision tree using the training data
decision_tree.train(pd.concat([X_train, y_train], axis=1), target_attribute)

# Predict the target attribute for the testing data
predictions = decision_tree.predict(pd.concat([X_test, y_test], axis=1))

# Evaluate the model using accuracy, precision, recall, and F1-score
accuracy = accuracy_score(y_test, predictions)
eval_accuracy = evaluation_metric.accuracy_score(y_test, predictions)

precision = precision_score(y_test, predictions, average='weighted')
eval_precision = evaluation_metric.precision_score(y_test, predictions, positive_label=1)

recall = recall_score(y_test, predictions, average='weighted')
eval_recall = evaluation_metric.recall_score(y_test, predictions, positive_label=1)

f1 = f1_score(y_test, predictions, average='weighted')
eval_f1 = evaluation_metric.f1_score(y_test, predictions, positive_label=1)

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Eval Accuracy: {eval_accuracy:.2f}")

print(f"Precision: {precision:.2f}")
print(f"Eval Precision: {eval_precision:.2f}")

print(f"Recall: {recall:.2f}")
print(f"Eval Recall: {eval_recall:.2f}")

print(f"F1-score: {f1:.2f}")
print(f"Eval F1-score: {eval_f1:.2f}")

decision_tree.print_tree()


# 2.2 Réseaux de neurones artificiels

### 1 - Division des données.

In [94]:
from sklearn.model_selection import train_test_split


def neural_split_data(data, target_attribute):

    # Séparez les données et les labels afin de pouvoir séparer en jeu d'entrainement
    # de test et de validation
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=[target_attribute]),
                                                        data[target_attribute], test_size=0.15, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15/0.85,
                                                    random_state=42)
    return X_train, y_train, X_val, y_val, X_test, y_test

#print (X_train)

In [95]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# Cross Entropy function.
def cross_entropy(y_pred, y_true):
 
    # computing softmax values for predicted values
    y_pred = softmax(y_pred)
    loss = 0
     
    # Doing cross entropy Loss
    for i in range(len(y_pred)):
 
        # Here, the loss is computed using the
        # above mathematical formulation.
        loss = loss + (-1 * y_true[i]*np.log(y_pred[i]))
 
    return loss
 
# y_true: True Probability Distribution
y_true = [1, 0, 0, 0, 0]
 
# y_pred: Predicted values for each calss
y_pred = [10, 5, 3, 1, 4]

# Calling the cross_entropy function by passing
# the suitable values
cross_entropy_loss = cross_entropy(y_pred, y_true)
 
print("Cross Entropy Loss: ", cross_entropy_loss)


print("Debugging Entropy Calculation:")
debug_data = pd.DataFrame({"Class": [1, 1, 1, 0, 0, 2, 2]})
print(f"Calculated Entropy: {entropie(debug_data, 'Class')}")

Cross Entropy Loss:  0.010199795719758053
Debugging Entropy Calculation:
Calculated Entropy: 1.556656707462822


### 2 - Création de la classe "NeuralNetwork"

In [96]:
import numpy as np

class NeuralNetwork:
    def __init__(self, architecture, activation='relu'):
        self.architecture = architecture
        self.activation = activation
        self.weights = []
        self.biases = []
        self.initialize_parameters()

    def initialize_parameters(self):
        np.random.seed(42)
        for i in range(1, len(self.architecture)):
            if self.activation == 'relu':
                # He initialization for ReLU
                weight_matrix = np.random.randn(self.architecture[i], self.architecture[i - 1]) * np.sqrt(2. / self.architecture[i - 1])
            else:
                # Xavier initialization for other activations
                weight_matrix = np.random.randn(self.architecture[i], self.architecture[i - 1]) * np.sqrt(1. / self.architecture[i - 1])
            bias_vector = np.zeros((self.architecture[i], 1))
            self.weights.append(weight_matrix)
            self.biases.append(bias_vector)

    def activation_function(self, x):
        if self.activation == 'relu':
            return np.maximum(0, x)
        elif self.activation == 'tanh':
            return np.tanh(x)
        else:
            raise ValueError("Unsupported activation function. Choose 'relu' or 'tanh'.")

    def activation_derivative(self, x):
        if self.activation == 'relu':
            return (x > 0).astype(float)
        elif self.activation == 'tanh':
            return 1 - np.tanh(x)**2
        else:
            raise ValueError("Unsupported activation function for derivative. Choose 'relu' or 'tanh'.")

    def forward_propagation(self, X):
        activations = [X.T]
        weighted_inputs = []
        for i in range(len(self.weights)):
            weighted_input = np.dot(self.weights[i], activations[-1]) + self.biases[i]
            weighted_inputs.append(weighted_input)
            activations.append(self.activation_function(weighted_input))
        return activations, weighted_inputs
    


    def backward_propagation(self, X, y, activations, weighted_inputs):
        m = X.shape[1]
        gradients = [np.zeros_like(w) for w in self.weights]
        delta = activations[-1] - y
        for i in range(m - 1, 0, -1):
            gradients[i] = np.dot(delta, activations[i - 1].T) / m
            delta = np.dot(self.weights[i].T, delta) * self.activation_derivative(weighted_inputs[i - 1])
        gradients[0] = np.dot(delta, X.T) / m
        return gradients
    
    def update_parameters(self, gradients, learning_rate):
        for i in range(len(self.weights)):
            self.weights[i] -= learning_rate * gradients[i]
            self.biases[i] -= learning_rate * gradients[i]
    
    def train(self, X, y, epochs=1000, learning_rate=0.01):
        for epoch in range(epochs):
            activations, weighted_inputs = self.forward_propagation(X)
            gradients = self.backward_propagation(X, y, activations, weighted_inputs)
            self.update_parameters(gradients, learning_rate)
            if epoch % 100 == 0:
                loss = cross_entropy(activations[-1], y)
                print(f"Epoch {epoch}, Loss: {loss}")

    def predict(self, X):
        activations, _ = self.forward_propagation(X)
        return activations[-1]
    
    def evaluate(self, X, y):
        predictions = self.predict(X)
        loss = cross_entropy(predictions, y)
        return loss
    
    def print_parameters(self):
        for i in range(len(self.weights)):
            print(f"Layer {i + 1}:")
            print("Weights:")
            print(self.weights[i])
            print("Biases:")
            print(self.biases[i])

# Define the target attribute
target_attribute = "Class"

# Split the data into training, validation, and testing sets using the neural_split_data function
X_train, y_train, X_val, y_val, X_test, y_test = neural_split_data(data, target_attribute)

# Define the architecture of the neural network
architecture = [X_train.shape[0], 64, 32, 4]

# Instantiate the NeuralNetwork class
neural_network = NeuralNetwork(architecture, activation='relu')

# Train the neural network using the training data
neural_network.train(X_train, y_train, epochs=1000, learning_rate=0.01)

# Evaluate the neural network using the validation data
validation_loss = neural_network.evaluate(X_val, y_val)
print(f"Validation Loss: {validation_loss}")

# Print the parameters of the neural network
neural_network.print_parameters()

# Evaluate the neural network using the testing data
test_loss = neural_network.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

# Predict the target attribute for the testing data
predictions = neural_network.predict(X_test)
print(predictions)

# Evaluate the model using accuracy, precision, recall, and F1-score
accuracy = accuracy_score(y_test, predictions)
eval_accuracy = evaluation_metric.accuracy_score(y_test, predictions)

precision = precision_score(y_test, predictions, average='weighted')
eval_precision = evaluation_metric.precision_score(y_test, predictions, positive_label=1)

recall = recall_score(y_test, predictions, average='weighted')
eval_recall = evaluation_metric.recall_score(y_test, predictions, positive_label=1)

f1 = f1_score(y_test, predictions, average='weighted')
eval_f1 = evaluation_metric.f1_score(y_test, predictions, positive_label=1)

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Eval Accuracy: {eval_accuracy:.2f}")

print(f"Precision: {precision:.2f}")
print(f"Eval Precision: {eval_precision:.2f}")

print(f"Recall: {recall:.2f}")
print(f"Eval Recall: {eval_recall:.2f}")

print(f"F1-score: {f1:.2f}")
print(f"Eval F1-score: {eval_f1:.2f}")  



ValueError: shapes (64,1608) and (14,1608) not aligned: 1608 (dim 1) != 14 (dim 0)

In [88]:
 # Spécifiez le nom de la colonne cible
target_attribute = "Class"

# Divisez les données en ensembles d'entraînement, de validation et de test
X_train, y_train, X_val, y_val, X_test, y_test = neural_split_data(
    data, target_attribute)

test_net = NeuralNetwork(architecture=[14, 8, 4], activation='relu',X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test,learning_rate=0.01,epoch=300)

### 3. Entraînement

In [ ]:
def train_models(X_train, y_train, X_val, y_val, architectures, activations, learning_rate=0.01, epochs=100, patience=4, batch_size=4):
    """
    Train multiple neural network models with specified architectures and activations.

    Parameters:
    - X_train (np.array): Training data.
    - y_train (np.array): Training labels.
    - X_val (np.array): Validation data.
    - y_val (np.array): Validation labels.
    - architectures (list of lists): List of architectures (list of integers) to train.
    - activations (list of str): List of activation functions to use ('tanh' and 'relu').
    - learning_rate (float): Learning rate for training.
    - epochs (int): Number of epochs to train for.
    - patience (int): Patience for early stopping.
    - batch_size (int): Size of mini-batch for training.

    Returns:
    - dict: Dictionary containing trained models for each architecture and activation function.
    """
    models = {}

    # Train models for each architecture and activation function
    for activation in activations:
        for architecture in architectures:
            # Create neural network model
            model = NeuralNetwork(architecture, activation)

            # Train the model
            model.train(X_train, y_train, X_val, y_val, learning_rate=learning_rate,
                        epochs=epochs, patience=patience, batch_size=batch_size)

            # Store the trained model
            models[f"{activation}_{architecture}"] = model

    return models

In [ ]:
architectures = [(14, 10, 8, 6, 4), (14, 10, 8, 4, 4), (14, 6, 4, 4)]
activations = ["tanh", "relu"]
models = train_models(X_train, y_train, X_val, y_val, architectures, activations)

### 4. Evaluation

In [ ]:
def evaluate_models(models, X_test, y_test):
    """
    Evaluate trained models on the test data.

    Parameters:
    - models (dict): Dictionary containing trained models.
    - X_test (np.array): Test data.
    - y_test (np.array): Test labels.
    
    Returns:
    - dict: Dictionary containing predictions and evaluation metrics for each model.
    """
    results = {}
    
    # Evaluate each model
    for model_name, model in models.items():
        # Predict the test data
        predictions = model.predict(X_test)
        
        # Calculate loss (or other evaluation metrics)
        
        loss = model.calculate_loss(X_test, y_test)

        accuracy = np.mean(predictions == y_test)

        
        # Store results
        results[model_name] = {
            "predictions": predictions,
            "loss": loss,
            "accuracy": accuracy
        }
        
    return results

In [ ]:
results = evaluate_models(models, X_test, y_test)
results

In [ ]:
# Spécifiez le nom de la colonne cible
target_attribute = "Class"

# Divisez les données en ensembles d'entraînement, de validation et de test
X_train, y_train, X_val, y_val, X_test, y_test = neural_split_data(data, target_attribute)


# Liste des architectures à tester
architectures = [(14, 10, 8, 6, 4)]

# Fonction d'activation à tester
activation = 'tanh'  # Utilisez 'tanh' ou 'relu'

neural_network = NeuralNetwork(architecture=architectures[0])
neural_network.train(X_train, y_train, X_val, y_val, learning_rate=0.01, epochs=50, patience=4, batch_size=4)


In [ ]:
# Charger les données
data = pd.read_csv('test_data.csv')

# Séparer les features et les labels
X = data.drop('Label', axis=1)
y = data['Label']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Initialiser le réseau de neurones
nn = NeuralNetwork(architecture=[(14, 10, 8, 6, 4)], activation='relu')  # Architecture avec 3 couches

In [ ]:
# Entraîner le réseau
nn.train(X_train, y_train, X_test, y_test, learning_rate=0.01, epochs=100, batch_size=4)


In [ ]:
# Prédire les labels sur l'ensemble de test
predictions = nn.predict(X_test)  # Assurez-vous que les dimensions correspondent aux attentes de votre classe
# Calculer les métriques de performance
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)
